# All treatment times

Calculate the treatment times for every Small Area for each possible stroke unit.

## Aims

The results a table with:
+ one row for each Small Area
+ columns for time to IVT and time to MT for each stroke unit attended

## Method

load in fixed pathway times

add on to travel times from each small area to the unit

add on transfer times if necessary

so calculate total time to IVT and to MT for each Small Area

## Code setup

In [4]:
import pandas as pd
import os

from dataclasses import dataclass

In [23]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    # Directories:
    dir_data = 'data'
    dir_output = 'output'

    # Reference data:
    units = 'hospitals.csv'
    sa_unit_travel_matrix = 'travel_matrix.csv'
    inter_unit_travel_matrix = 'inter_hospital_time.csv'
    # Pathway timings:
    fixed_times = 'pathway_times.csv'

    # Output files:
    df_treatment_times = 'sa_treatment_times.csv'

paths = Paths()

## Generic treatment times without travel

Load fixed pathway timings:

In [12]:
path_to_timings = os.path.join(paths.dir_data, paths.fixed_times)
series_fixed_times = pd.read_csv(path_to_timings, index_col=0, header=None).squeeze()
series_fixed_times.name = 'fixed_times_mins'

series_fixed_times

0
onset_to_ambulance_arrival                  60
arrival_to_ivt                              40
arrival_to_mt                               90
net_operational_delay_to_mt_for_transfer    60
Name: fixed_time, dtype: int64

Combine these fixed times to calculate the treatment times before adding on any travel times.

In [13]:
# IVT:
series_fixed_times['ivt_without_travel'] = (
    series_fixed_times['onset_to_ambulance_arrival'] +
    series_fixed_times['arrival_to_ivt']
)

# MT without transfer:
series_fixed_times['mt_without_travel'] = (
    series_fixed_times['onset_to_ambulance_arrival'] +
    series_fixed_times['arrival_to_mt']
)

# MT with transfer:
series_fixed_times['mt_transfer_without_travel'] = (   
    series_fixed_times['onset_to_ambulance_arrival'] +
    series_fixed_times['net_operational_delay_to_mt_for_transfer'] +
    series_fixed_times['arrival_to_mt']
)

View the results:

In [14]:
series_fixed_times

0
onset_to_ambulance_arrival                   60
arrival_to_ivt                               40
arrival_to_mt                                90
net_operational_delay_to_mt_for_transfer     60
ivt_without_travel                          100
mt_without_travel                           150
mt_transfer_without_travel                  210
Name: fixed_time, dtype: int64

## Load travel data

Stroke unit services:

In [18]:
path_to_units = os.path.join('..', paths.dir_data, paths.units)
df_units = pd.read_csv(path_to_units, index_col=0)

df_units

,Hospital_name,City,door_to_needle,Fixed,use_ivt,use_mt
Postcode,,,,,,
BT126BA,Royal Victoria,Belfast,43,2.0,1,1
BT161RH,Ulster,Belfast,50,NaN,1,0
BT358DR,Daisy Hill,Newry,38,NaN,1,0
BT412RL,Antrim Area,Antrim,59,NaN,1,0
BT476SB,Altnagelvin,Londonderry,68,NaN,1,0
BT521HS,Causeway,Coleraine,45,NaN,1,0
BT635QQ,Craigavon Area,Craigavon,38,NaN,1,0
BT746DN,South West,Enniskillen,53,NaN,1,0


Travel matrix between each Small Area and each stroke unit:

In [7]:
path_to_sa_matrix = os.path.join('..', paths.dir_data, paths.sa_unit_travel_matrix)
df_travel_sa_unit = pd.read_csv(path_to_sa_matrix, index_col=0)

df_travel_sa_unit.head()

,BT126BA,BT161RH,BT358DR,BT412RL,BT476SB,BT521HS,BT635QQ,BT746DN
from_postcode,,,,,,,,
N00000001,31.400562,37.852665,64.223641,19.032658,79.895643,61.926859,36.580462,102.7
N00000002,24.677730,37.852665,61.926859,23.292278,83.200000,65.366046,34.009674,100.1
N00000003,24.677730,37.852665,63.077279,20.471335,80.991077,63.077279,35.299611,101.4
N00000004,30.080264,42.862547,66.504588,24.677730,83.200000,65.366046,39.116622,105.3
N00000005,32.710168,40.372700,66.504588,16.088440,76.591575,58.450174,40.372700,106.6


Travel matrix between each stroke unit and each other stroke unit:

In [9]:
path_to_unit_matrix = os.path.join('..', paths.dir_data, paths.inter_unit_travel_matrix)
df_travel_units = pd.read_csv(path_to_unit_matrix, index_col=0)

df_travel_units

,BT126BA,BT161RH,BT358DR,BT412RL,BT476SB,BT521HS,BT635QQ,BT746DN
from_postcode,,,,,,,,
BT126BA,0.000000,19.032658,52.563615,27.404887,88.400000,69.897939,39.116622,105.300000
BT161RH,19.032658,0.000000,63.077279,35.299611,96.200000,77.695950,50.173458,117.000000
BT358DR,51.371234,63.077279,0.000000,69.897939,117.000000,114.400000,35.299611,94.900000
BT412RL,28.748618,35.299611,71.021910,0.000000,72.142439,53.750768,48.970106,110.500000
BT476SB,88.400000,96.200000,117.000000,72.142439,0.000000,45.324617,98.800000,85.800000
BT521HS,69.897939,76.591575,114.400000,53.750768,45.324617,0.000000,88.400000,122.200000
BT635QQ,37.852665,50.173458,35.299611,48.970106,98.800000,88.400000,0.000000,79.895643
BT746DN,104.000000,117.000000,94.900000,115.700000,85.800000,122.200000,80.991077,0.000000


## Calculate treatment times for Small Areas

Assume that all stroke units transfer patients to the Royal Victoria (BT126BA) if necessary for MT.

In [19]:
df_treatment_times = pd.DataFrame(index=df_travel_sa_unit.index)

for first_stroke_unit in df_travel_sa_unit.columns:
    # Check whether this unit provides MT.
    if df_units.loc[first_stroke_unit, 'use_mt'] == 1:
        # No transfer required for MT.
        time_to_mt_without_travel = series_fixed_times['mt_without_travel']
    else:
        # Transfer required for MT.
        time_to_mt_without_travel = series_fixed_times['mt_transfer_without_travel']
    # Travel time from this unit to the Royal Victoria:
    # (if the first unit _is_ the Royal Victoria, this time will be zero)
    transfer_travel_time = df_travel_units.loc[first_stroke_unit, 'BT126BA']

    # Calculate times to treatment including travel:
    df_treatment_times[f'{first_stroke_unit}_ivt'] = (
        series_fixed_times['ivt_without_travel'] + 
        df_travel_sa_unit[first_stroke_unit]
    )
    df_treatment_times[f'{first_stroke_unit}_mt'] = (
        time_to_mt_without_travel + 
        df_travel_sa_unit[first_stroke_unit] +
        transfer_travel_time
    )
    

In [20]:
df_treatment_times.head()

,BT126BA_ivt,BT126BA_mt,BT161RH_ivt,BT161RH_mt,BT358DR_ivt,BT358DR_mt,BT412RL_ivt,BT412RL_mt,BT476SB_ivt,BT476SB_mt,BT521HS_ivt,BT521HS_mt,BT635QQ_ivt,BT635QQ_mt,BT746DN_ivt,BT746DN_mt
from_postcode,,,,,,,,,,,,,,,,
N00000001,131.400562,181.400562,137.852665,266.885323,164.223641,325.594875,119.032658,257.781276,179.895643,378.295643,161.926859,341.824797,136.580462,284.433127,202.7,416.7
N00000002,124.677730,174.677730,137.852665,266.885323,161.926859,323.298093,123.292278,262.040896,183.200000,381.600000,165.366046,345.263984,134.009674,281.862339,200.1,414.1
N00000003,124.677730,174.677730,137.852665,266.885323,163.077279,324.448513,120.471335,259.219953,180.991077,379.391077,163.077279,342.975217,135.299611,283.152276,201.4,415.4
N00000004,130.080264,180.080264,142.862547,271.895205,166.504588,327.875822,124.677730,263.426348,183.200000,381.600000,165.366046,345.263984,139.116622,286.969287,205.3,419.3
N00000005,132.710168,182.710168,140.372700,269.405358,166.504588,327.875822,116.088440,254.837059,176.591575,374.991575,158.450174,338.348112,140.372700,288.225365,206.6,420.6


Save times to file:

In [25]:
df_treatment_times.to_csv(os.path.join(paths.dir_output, paths.df_treatment_times))